In [2]:
%matplotlib inline
import matplotlib.pyplot as pl
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
import seaborn as sns

import numpy as np
import pandas as pd
from tqdm import tqdm

# A few useful constants
KEPLER_BEGIN_BK, KEPLER_END_BK = 130, 1582

In [3]:
# Read the TCE table
tcedf = pd.read_csv('../data/q1_q17_dr25_tce.csv')
# Add the unique TCE ID used by the RoboVetter output:
tcedf.loc[:, 'tce'] = ['{:09d}-{:02d}'.format(row.kepid, row.tce_plnt_num) for row in tcedf.itertuples()]

In [12]:
mask = tcedf.tce_period > 100
transitrows = []
for mytce in tqdm(tcedf[mask].itertuples()):
    mytime = mytce.tce_time0bk
    while mytime < KEPLER_END_BK:
        newrow = {'transit_time': mytime,
                  'tce': mytce.tce,
                  'kepid': mytce.kepid,
                  'tce_plnt_num': mytce.tce_plnt_num,
                  'tce_period': mytce.tce_period,
                  'tce_max_mult_ev': mytce.tce_max_mult_ev}                  
        transitrows.append(newrow)
        mytime += mytce.tce_period
transits = pd.DataFrame(transitrows)
transits.to_hdf('all-long-period-tce-transits.h5', key='transits')
len(transits)

66047

In [6]:
KEPLER_QUARTERS = pd.read_csv('../data/kepler-quarters.csv')

def mjd2quarter(mjd):
    mask = (KEPLER_QUARTERS.first_lc_mjd < mjd+0.01) & (KEPLER_QUARTERS.last_lc_mjd > mjd-0.01)
    if mask.any():
        return KEPLER_QUARTERS.loc[mask, 'quarter'].values[0]
    return None

def mjd2season(mjd):
    mask = (KEPLER_QUARTERS.first_lc_mjd < mjd+0.01) & (KEPLER_QUARTERS.last_lc_mjd > mjd-0.01)
    if mask.any():
        return KEPLER_QUARTERS.loc[mask, 'season'].values[0]
    return None

def bkjd_to_mjd_approximate(bkjd):
    """Inexact conversion from Barycentric Kepler Julian Date (BKJD) to Modified Julian Date (MJD).
    
    Inexact because it ignores the TIMECORR and TIMSLICE corrections.
    """
    return bkjd + 2454833 - 2400000.5

def bkjd2quarter(bkjd):
    return mjd2quarter(bkjd_to_mjd_approximate(bkjd))

In [22]:
# Add a column detailing the quarter
quarter_column, season_column = [], []
for row in tqdm(transits.itertuples()):
    mjd = bkjd_to_mjd_approximate(row.transit_time)
    quarter_column.append(mjd2quarter(mjd))
    season_column.append(mjd2season(mjd))
transits['quarter'] = quarter_column
transits['season'] = season_column

In [8]:
import requests

def get_ccdinfo_from_mast(kepler_id, transit_time):
    """Obtain the CCD channel, module, and skygroup for a given kepid/time from MAST."""
    quarter = mjd2quarter(bkjd_to_mjd_approximate(transit_time))
    try:
        q = int(quarter)
    except Exception:
        return {'channel': None, 'module': None, 'output': None, 'skygroup': None}
    max_records = np.random.randint(12345, 1234567890)  # Hack to prevent MAST from throwing a stupid 'Max retries exceeded with url' error
    url = ('http://archive.stsci.edu/kepler/data_search/search.php?'
           'target={}&sci_data_quarter={}'
           '&action=Search&outputformat=JSON'
           '&max_records={}').format(int(kepler_id), int(quarter), max_records)
    resp = requests.get(url)
    if 'no rows found' in str(resp.content):
        return {'channel': None, 'module': None, 'output': None, 'skygroup': None}
    else:
        return {'channel': int(resp.json()[0]['Channel']),
                'module': int(resp.json()[0]['Module']),
                'output': int(resp.json()[0]['Output']),
                'skygroup': int(resp.json()[0]['Skygroup_ID'])}

def get_ccdinfo(transit_idx):
    mytransit = transits.ix[transit_idx]
    ccdinfo = get_ccdinfo_from_mast(mytransit.kepid, mytransit.transit_time)
    ccdinfo['idx'] = transit_idx
    ccdinfo['kepid'] = mytransit.kepid
    ccdinfo['transit_time'] = mytransit.transit_time
    return ccdinfo

In [9]:
get_ccdinfo(10)

{'channel': 32,
 'idx': 10,
 'kepid': 892376,
 'module': 10,
 'output': 4,
 'skygroup': 84,
 'transit_time': 498.041}

In [10]:
import multiprocessing
pool = multiprocessing.Pool(processes=20)
results = []
for result in tqdm(pool.imap(get_ccdinfo, transits.index),
                   total=len(transits), mininterval=120, maxinterval=240):
    results.append(result)
pool.close()

In [19]:
len(results)

66047

In [35]:
results_copy = results

In [11]:
len(results)

66047

In [13]:
ccdinfo = pd.DataFrame(results)
ccdinfo.index = ccdinfo['idx']
ccdinfo.head()

,channel,idx,kepid,module,output,skygroup,transit_time
idx,,,,,,,
0,4.0,0,2304168,2.0,4.0,84.0,219.298
1,56.0,1,2304168,16.0,4.0,84.0,651.017
2,56.0,2,2304168,16.0,4.0,84.0,1082.736
3,84.0,3,2304168,24.0,4.0,84.0,1514.455
4,32.0,4,2303102,10.0,4.0,84.0,161.570


In [24]:
transits_with_ccdinfo = transits.merge(ccdinfo, left_index=True, right_index=True)
len(transits_with_ccdinfo)

66047

In [25]:
mask = transits_with_ccdinfo.channel.isnull()
transits_with_ccdinfo[mask]

,kepid_x,tce,tce_max_mult_ev,tce_period,tce_plnt_num,transit_time_x,quarter,season,channel,idx,kepid_y,module,output,skygroup,transit_time_y
26,893507,000893507-02,14.160,302.046,2,891.079,9.0,3.0,NaN,26,893507,NaN,NaN,NaN,891.079
27,893507,000893507-02,14.160,302.046,2,1193.125,13.0,3.0,NaN,27,893507,NaN,NaN,NaN,1193.125
36,893507,000893507-03,15.010,300.228,3,1221.107,13.0,3.0,NaN,36,893507,NaN,NaN,NaN,1221.107
39,893507,000893507-04,13.680,285.254,4,536.772,5.0,3.0,NaN,39,893507,NaN,NaN,NaN,536.772
40,893507,000893507-04,13.680,285.254,4,822.026,9.0,3.0,NaN,40,893507,NaN,NaN,NaN,822.026
45,893507,000893507-05,11.330,308.893,5,889.176,9.0,3.0,NaN,45,893507,NaN,NaN,NaN,889.176
46,893507,000893507-05,11.330,308.893,5,1198.069,13.0,3.0,NaN,46,893507,NaN,NaN,NaN,1198.069
55,893507,000893507-06,11.580,279.221,6,1206.366,13.0,3.0,NaN,55,893507,NaN,NaN,NaN,1206.366
61,893647,000893647-01,12.450,407.102,1,725.497,NaN,NaN,NaN,61,893647,NaN,NaN,NaN,725.497
78,1026133,001026133-02,10.990,186.181,2,802.975,NaN,NaN,NaN,78,1026133,NaN,NaN,NaN,802.975


In [26]:
transits_with_ccdinfo.head()

,kepid_x,tce,tce_max_mult_ev,tce_period,tce_plnt_num,transit_time_x,quarter,season,channel,idx,kepid_y,module,output,skygroup,transit_time_y
0,2304168,002304168-02,12.22,431.719,2,219.298,2.0,0.0,4.0,0,2304168,2.0,4.0,84.0,219.298
1,2304168,002304168-02,12.22,431.719,2,651.017,7.0,1.0,56.0,1,2304168,16.0,4.0,84.0,651.017
2,2304168,002304168-02,12.22,431.719,2,1082.736,11.0,1.0,56.0,2,2304168,16.0,4.0,84.0,1082.736
3,2304168,002304168-02,12.22,431.719,2,1514.455,16.0,2.0,84.0,3,2304168,24.0,4.0,84.0,1514.455
4,2303102,002303102-10,11.51,480.481,10,161.570,1.0,3.0,32.0,4,2303102,10.0,4.0,84.0,161.570


In [30]:
mask_not_observed = transits_with_ccdinfo['channel'].isnull()
print('{} transits were not observed'.format(mask_not_observed.sum()))

5140 transits were not observed


In [49]:
observed_transits = transits_with_ccdinfo[~mask_not_observed].copy()
for column in ['quarter', 'season', 'channel', 'module', 'output', 'skygroup']:
    observed_transits[column] = observed_transits[column].astype('int')

In [50]:
observed_transits.head()

,kepid_x,tce,tce_max_mult_ev,tce_period,tce_plnt_num,transit_time_x,quarter,season,channel,idx,kepid_y,module,output,skygroup,transit_time_y
0,2304168,002304168-02,12.22,431.719,2,219.298,2,0,4,0,2304168,2,4,84,219.298
1,2304168,002304168-02,12.22,431.719,2,651.017,7,1,56,1,2304168,16,4,84,651.017
2,2304168,002304168-02,12.22,431.719,2,1082.736,11,1,56,2,2304168,16,4,84,1082.736
3,2304168,002304168-02,12.22,431.719,2,1514.455,16,2,84,3,2304168,24,4,84,1514.455
4,2303102,002303102-10,11.51,480.481,10,161.570,1,3,32,4,2303102,10,4,84,161.570


In [55]:
assert((observed_transits.kepid_x == observed_transits.kepid_y).all())
assert((observed_transits.transit_time_x == observed_transits.transit_time_y).all())

In [56]:
len(observed_transits)

60907

In [58]:
observed_transits.to_hdf('observed-long-period-transits.h5', key='transits')